In [1]:
import requests
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./app/Qwen2-1.5B-Instruct-bnb-4bit")

def send_request(text):
    url = "http://0.0.0.0:8080/"
    data = text
    response = requests.post(url, data=data, stream=True)
    return response

def generate_word(conv:list, input_text:str):
    conv.append(dict(role="user", content=input_text))

    chat = tokenizer.apply_chat_template(
        conv, tokenize=False, add_generation_prompt=True
    )

    generated_text = '' 
    response = send_request(chat)
    for chunk in response.iter_content(None):
        new_text = chunk.decode('utf-8')
        yield new_text
        generated_text += new_text
        
    conv.append(dict(role="assistant", content=generated_text))


conv = []

while True:
    input_text = input("INPUT: ")
    for word in generate_word(conv, input_text):
        print(word, end='', flush=True)
    print()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Hello! How can I assist you today?


In [2]:
import requests
# from starlette import requests

def send_request(text):
    url = "http://0.0.0.0:8080/"
    # data = text
    response = requests.post(url, data=text, stream=True)
    print(response)
    print(type(response))
    return response.text

# while True:
#     url = "http://0.0.0.0:8080/"
#     data = input("INPUT: ")
#     response = Request(url, data=data)
    
requests.re

In [ ]:
conv = []

while True:
    conv.append(dict(role="user", content=input("Input: ")))

    chat = tokenizer.apply_chat_template(
        conv, tokenize=False, add_generation_prompt=True
    )

    result = send_request(chat)
    
    # print(result[0]["generated_text"])

    # conv.append(dict(role="assistant", content=result[0]["generated_text"]))


In [1]:
import requests
import json

def client():
    url = "http://localhost:8000/"  # replace with the actual server URL
    data = {"text": [
        {'role': 'user', "content":'Hello'},
        ]}  # replace with the actual input text

    response = requests.post(url, json=data, stream=True)

    if response.status_code == 200:
        for line in response.iter_lines():
            if line:
                print(json.loads(line)["word"])  # assuming the server returns a JSON object with a "word" key
    else:
        print("Error:", response.status_code)

if __name__ == "__main__":
    client()

RuntimeError: asyncio.run() cannot be called from a running event loop

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer, pipeline
from threading import Thread

tokenizer = AutoTokenizer.from_pretrained("./app/Qwen2-1.5B-Instruct-bnb-4bit")
model = AutoModelForCausalLM.from_pretrained("./app/Qwen2-1.5B-Instruct-bnb-4bit", device_map = 'auto', torch_dtype = 'auto')
pipe = pipeline('text-generation', model = model, tokenizer=tokenizer)
streamer = TextIteratorStreamer(tokenizer, skip_prompt=True)

# Run the generation in a separate thread, so that we can fetch the generated text in a non-blocking way.
conv = []
while True:
    conv.append(dict(role = 'user', content = input()))
    input_ids = tokenizer.apply_chat_template(conv, return_tensors="pt", add_generation_prompt=True, tokenize=False)
    thread = Thread(target=pipe, kwargs=dict(text_inputs= input_ids, max_new_tokens = 512, streamer = streamer))
    thread.start()
    generated_text = ""
    for new_text in streamer:
        print(new_text, end = '', flush = True)
        generated_text += new_text
        
    print("\n")
    conv.append(dict(role = 'assistant', content = generated_text))
    # generated_text


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Hello! How can I assist you today?<|im_end|>

I'm an AI language model designed to answer questions and provide information on various topics. How may I help you today?<|im_end|>

As an artificial intelligence, my primary goal is to provide assistance and support to users in order to make their lives easier and more efficient. By answering questions, providing information, and assisting with tasks, I can help people achieve their goals and improve their overall experience. Additionally, as part of my programming, I am constantly learning and improving, so any feedback or suggestions from users will help me become even better at serving them.<|im_end|>

